In [7]:
# number of overdues 6, 12 months
# overdue amount past 6, 12 months
# total sanction amount of ongoing loans
# highest sanction amount of ongoing loans
# total number of ongoing loans
# total emi amount of current ongoing loans
# number of settlements past 6, 12 months
# total current balance of ongoing loans

In [76]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt

#path = r"C://Users/Alok169231/Downloads/project_new/project_new-master/"

In [77]:
tempdf = pd.read_csv("loan_type_sec_unsec.csv")

In [78]:
df = tempdf.copy()
df = df[df['cibil__user'].notnull()]
df['cibil__user'] = df['cibil__user'].astype(int).astype(object)

# overdue logic
last_day = dt(2021,9,30)
date_in_past = last_day - datetime.timedelta(days=365)
df.closed_date = pd.to_datetime(df.closed_date)

def overdue(row, date_in_past): # date_in_past can be anytime in past i.e. 6, 12, 18, 24, 30, 36 months or even more.
    try:
        if pd.isnull(row.closed_date) or (row.closed_date >= date_in_past):
            if row.overdue !=0:
                return 1
    except:
        print("Exception Raised !")
    return 0

df['overdue_12month'] = df.apply(lambda row: overdue(row, date_in_past), axis=1)
df = df.groupby('cibil__user').apply(lambda row: pd.Series(dict(
    overdue_amount_ongoing = row[row.overdue_12month == 1].overdue.sum(),
    overdue_number_ongoing = row[row.overdue_12month == 1].overdue_12month.sum(),
    funded_amount_ongoing = row[pd.isnull(row.closed_date)].sanctioned.sum(),
    highest_fundamt_ongoing = row[pd.isnull(row.closed_date)].sanctioned.max(),
    total_loans_ongoing = row[pd.isnull(row.closed_date)&(row.current_balance>0)].cibil__user.count(),
    total_emi_ongoing = row[pd.isnull(row.closed_date)].emi_amount.sum(),
    total_current_due_ongoing = row[pd.isnull(row.closed_date)].current_balance.sum(),
    settle_12month = row[pd.isnull(row.closed_date)|(row.closed_date>=date_in_past)].settled.sum()
)))

In [81]:
df = df.reset_index()

In [83]:
df.to_csv("loan_type_derived_feature.csv",index=False)